# Import necessary modules

We begin by importing all necessary modules. The install function can be used to install the uninstalled modules.

In [7]:
import sys
import os
sys.path.append(os.path.abspath('../src'))
import urllib3
import json
import pandas as pd
import psycopg2
from sqlalchemy import create_engine, text

# Conect to PosgreSQL and Update data

In [8]:
# PostgreSQL connection info
username = "postgres"  # or your PostgreSQL username
password = "1234"
host = "localhost"           # or e.g. "127.0.0.1"
port = "5432"                # default port
database = "energy_forecast"  # or your database name

# Create connection engine
engine = create_engine(f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}")

conn = engine.connect()
result = conn.execute(text("SELECT MAX(\"DATETIME\") FROM public.generation_data;"))
last_row = result.fetchall()[0][0]

http = urllib3.PoolManager()
query_url = "https://api.neso.energy/api/3/action/datastore_search_sql?sql=SELECT * from \"f93d1835-75bc-43e5-84ad-12472b180a98\" " \
"WHERE \"DATETIME\" > '" + str(last_row) + "'"
new_resp = http.request("GET", query_url)
data = pd.DataFrame(json.loads(new_resp.data)['result']['records'])

# Preprocess dataframe

In [9]:
if not data.empty:
    cols = data.columns.tolist()
    non_perc_cols = [col for col in cols if not col.endswith('_perc')]
    perc_cols = [col for col in cols if col.endswith('_perc')]
    cols = non_perc_cols + perc_cols
    remove = ['DATETIME', 'CARBON_INTENSITY', '_full_text', '_id']
    # cols = ['DATETIME', 'CARBON_INTENSITY'] + [col for col in cols if col not in remove]
    cols = ['CARBON_INTENSITY'] + [col for col in cols if col not in remove]
    data_sort= pd.concat([
        data['DATETIME'].astype('datetime64[ns]'), 
        data[cols].astype(float)
        ], axis=1)
    # data_sort = data[cols]
    print(data_sort.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 34 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   DATETIME          50 non-null     datetime64[ns]
 1   CARBON_INTENSITY  50 non-null     float64       
 2   NUCLEAR           50 non-null     float64       
 3   STORAGE           50 non-null     float64       
 4   LOW_CARBON        50 non-null     float64       
 5   HYDRO             50 non-null     float64       
 6   WIND_EMB          50 non-null     float64       
 7   IMPORTS           50 non-null     float64       
 8   GAS               50 non-null     float64       
 9   COAL              50 non-null     float64       
 10  FOSSIL            50 non-null     float64       
 11  GENERATION        50 non-null     float64       
 12  OTHER             50 non-null     float64       
 13  ZERO_CARBON       50 non-null     float64       
 14  BIOMASS           50 non-nul

In [6]:
if not data.empty:
    data_sort.to_sql("generation_data", engine, if_exists="append", index=False)
    print("Data updated in PostgreSQL successfully.")

Data updated in PostgreSQL successfully.
